# Tarea 3: Derivación automática 1

Fecha de envío del PR inicial: **lunes 27 de marzo, antes de la clase**

Fecha de aceptación del PR: **martes 4 de abril, antes de la clase**

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que llamaremos `Dual` y que defina los duales, donde el parámetro debe ser un subtipo de `Real`; la siguiente celda sirve para empezar. La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

---

In [1]:
"""
    Dual{T<:Real}

Definición de los duales, donde lo campos son:

fun y der que corresponden a una función y su derivada respectivamente 
...
"""
type Dual{T<:Real}
    # código: 
    fun::T
    der::T
end

Dual

In [2]:
methods(Dual)

# 2 methods for generic function "(::Type)":
(::Type{Dual}){T<:Real}(fun::T, der::T) at In[1]:11
(::Type{T}){T}(arg) at sysimg.jl:53

In [3]:
Dual(1,2)

Dual{Int64}(1,2)

Si las entradas del dual no son del mismo tipo:

In [4]:
Dual(1, 2.4)

LoadError: MethodError: no method matching Dual{T<:Real}(::Int64, ::Float64)[0m
Closest candidates are:
  Dual{T<:Real}{T<:Real}(::T<:Real, [1m[31m::T<:Real[0m) at In[1]:11
  Dual{T<:Real}{T}(::Any) at sysimg.jl:53[0m

In [5]:
#= 
Definan un método que permita la promoción automática cuando 
las entradas para definir el dual no son del mismo tipo
=#

Dual(fun,der)= Dual(promote(fun,der)...)

Dual{T<:Real}

In [6]:
Dual(1, 2.4)

Dual{Float64}(1.0,2.4)

Si sólo hay una entrada en el dual:

In [7]:
Dual(4)

LoadError: MethodError: Cannot `convert` an object of type Int64 to an object of type Dual{T<:Real}
This may have arisen from a call to the constructor Dual{T<:Real}(...),
since type constructors fall back to convert methods.

In [8]:
#= 
Aquí se define un método que garantiza que el dual de una constante 
(número) cumple lo requerido
=#

Dual(f)=Dual(f,0)

Dual{T<:Real}

In [9]:
Dual(4)

Dual{Int64}(4,0)

In [10]:
Dual(4.5)

Dual{Float64}(4.5,0.0)

Finalmente, el dual tiene los siguientes métodos:

In [11]:
methods(Dual)

# 4 methods for generic function "(::Type)":
(::Type{Dual}){T<:Real}(fun::T, der::T) at In[1]:11
(::Type{Dual})(f) at In[8]:6
(::Type{Dual})(fun, der) at In[5]:6
(::Type{T}){T}(arg) at sysimg.jl:53

In [12]:
#= 
Aquí se define la función `xdual`, que se usará para identificar 
la variable independiente. La función dependerá de x_0, y debe 
regresar el Dual apropiado a la variable independiente
=#

"""
    xdual(x0) -> Dual(x0, 1)

...
"""
function xdual(x0)
    # código
    
    return Dual(x0,1)
    
end

xdual

In [13]:
xdual(9)

Dual{Int64}(9,1)

In [14]:
#= 
Muestren que su código funciona con tests adecuados para crear duales,
para promoverlos, y al definir el dual de un número y `xdual`.

En esto es útil usar la infraestructura de Julia; ver:
https://julia.readthedocs.io/en/stable/stdlib/test/

using Base.Test

a = Dual(1, 2.0)
@test a.fun == 1.0
@test a.der == 2.0

=#

using Base.Test

a = Dual(1, 2.0)

@testset "pruebas para el dual" begin
@test a.fun==1.0
@test a.der==2.0
    end    
    

Test Summary:        | Pass  Total
  pruebas para el dual |    2      2


Base.Test.DefaultTestSet("pruebas para el dual",Any[],2,false)

In [15]:
constante=Dual(4)
@test constante.fun ==4 

Test Passed
  Expression: constante.fun == 4
   Evaluated: 4 == 4

In [16]:
@test constante.der ==0

Test Passed
  Expression: constante.der == 0
   Evaluated: 0 == 0

In [17]:
constante2=xdual(9)

Dual{Int64}(9,1)

In [18]:
@test constante2.fun ==9

Test Passed
  Expression: constante2.fun == 9
   Evaluated: 9 == 9

In [19]:
@test constante2.der ==1

Test Passed
  Expression: constante2.der == 1
   Evaluated: 1 == 1

In [20]:
constante3=xdual(8)

Dual{Int64}(8,1)

In [21]:
@test constante3.fun == 8

Test Passed
  Expression: constante3.fun == 8
   Evaluated: 8 == 8

In [22]:
@test constante3.der == 1

Test Passed
  Expression: constante3.der == 1
   Evaluated: 1 == 1

---

## Ejercicio 2

Implementen *todas* las operaciones aritméticas definidas arriba. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`), como por ejemplo `a+b` o `b+a`, etc. Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). 

Implementen también la comparación entre duales (`==`). 

Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

Para operar duales con reales consideramos: $c=(c,0)$ donde $c$ es un real. 

In [23]:
import Base: +, -, *, /, ^, ==

#= 
Aquí se implementan los métodos necesarios para cada función; 
en el caso de ^ por ahora nos conformaremos con que la potencia 
sea entera.
=#

+(a::Dual, b::Dual) = Dual(a.fun+b.fun, a.der+b.der)
+(a::Dual, b::Real) = Dual(a.fun+b,a.der)
+(a::Real, b::Dual) = Dual(a+b.fun,b.der)

-(a::Dual, b::Dual) = Dual(a.fun-b.fun, a.der-b.der)
-(a::Dual, b::Real) = Dual(a.fun-b,a.der)
-(a::Real, b::Dual) = Dual(a-b.fun,-b.der)

*(a::Dual, b::Dual) = Dual(a.fun*b.fun, a.fun*b.der+b.fun*a.der)
*(a::Dual, b::Real) = Dual(b*a.fun,b*a.der)
*(a::Real, b::Dual) = Dual(a*b.fun,a*b.der)

/(a::Dual, b::Dual) = Dual(a.fun/b.fun, a.der-(a.fun/b.fun)*b.der/b.fun)
/(a::Dual, b::Real) = Dual( a.fun/b, a.der/b)

^(a::Dual, b::Int64) = Dual(a.fun^b,b*a.fun^(b-1)*a.der)

==(a::Dual, b::Dual) = a.fun==b.fun && a.der==b.der
==(a::Dual, b::Real) = a==b.fun && 0.0==b.der
==(a::Real, b::Dual) = a.fun==b && a.der==0.0


== (generic function with 121 methods)

Cálculo de la derivada.

Consideramos la siguiente función:

In [24]:
function f(x)
    return 3*x^3-2
end

f (generic function with 1 method)

Como queremos conocer el valor de la derivada cuando $x_0=1$

In [25]:
xdual(1.0)

Dual{Float64}(1.0,1.0)

Evaluamos lo anterior en la función $f$:

In [26]:
f(xdual(1))

Dual{Int64}(1,9)

Como podemos observar, el miembro derecho del dual es justo el valor de la derivada en $x_0=1$

In [27]:
# Aquí se incluyen las pruebas necesarias

using Base.Test


@testset "pruebas de operaciones para dual y xdual" begin
    @test Dual(4,1)+Dual(9,2)==Dual(13,3)
    @test Dual(4,1)+Dual(9)==Dual(13,1)    
    @test Dual(4)+Dual(9,2)==Dual(13,2)    
    @test Dual(4,1)-Dual(9,2)==Dual(-5,-1)    
    @test Dual(4,1)-Dual(9)==Dual(-5,1) 
    @test Dual(4)-Dual(9,2)==Dual(-5,-2)  
    @test Dual(4,1)*Dual(9,2)==Dual(36,17)
    @test Dual(4,1)*Dual(9)==Dual(36,9)
    @test Dual(4)*Dual(9,2)==Dual(36,8)
    @test Dual(4,2)/Dual(2,2)==Dual(2,0)
    @test Dual(4,2)/Dual(2)==Dual(2,2)
    @test Dual(2)/Dual(2,2)==Dual(1,-1)
    @test Dual(4,2)^2==Dual(16,16)
    @test Dual(4)==Dual(4,0)
    @test Dual(4.0)==Dual(4.0,0.0)
    @test Dual(4,0)==Dual(4)
    @test Dual(5)==Dual(5,0)
    @test Dual(8,2)==Dual(8,2)
    @test Dual(2,2)-4==Dual(-2,2)
    @test Dual(2,2)+4==Dual(6,2)
    @test 4-Dual(1,1)==Dual(3,-1)
    @test 4+Dual(2,2)==Dual(6,2)
    @test Dual(2,2)*4==Dual(8,8)
    @test 4*Dual(2,2)==Dual(8,8)
    @test Dual(2,2)/4==Dual(0.5,0.5)
    @test Dual(2,1)-2==Dual(0,1)
    @test Dual(2,1)/2==Dual(1.0,0.5)
    @test xdual(2) == Dual(2,1)
    @test xdual(2.0) == Dual(2,1)
    @test f(xdual(1))==Dual(1,9)
    
    
    end  


Test Summary:                            | Pass  Total
  pruebas de operaciones para dual y xdual |   30     30


Base.Test.DefaultTestSet("pruebas de operaciones para dual y xdual",Any[],30,false)

---

## Ejercicio 3

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo cuyo nombre será "AutomDiff.jl" y cuya estructura será la siguiente

```julia
#=
Aquí viene una explicación de lo que se hace en el módulo, 
los autores y la fecha
=#

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    ...

end
```

Todas las pruebas deberán ser incluidas en un archivo separado llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos
...

```

Estos dos archivos deben incluirlos en el envío de su tarea (además del archivo `Tarea3.ipynb`).

In [ ]:
__precompile__(true)

module AD
    import Base: +, -, *, /, ^, ==
    
    export Dual, xdual
    
    # Aquí viene TODO el código que implementaron.
    # Primero uno incluye la definición de Dual y
    # después las operaciones con Duales.
    """
    Dual{T<:Real}

Definición de los duales, donde lo campos son:

fun y der que corresponden a una función y su derivada respectivamente 
...
"""
type Dual{T<:Real}
    # código: 
    fun::T
    der::T
end

Dual(fun,der)= Dual(promote(fun,der)...)
Dual(f)=Dual(f,0)


"""
    xdual(x0) -> Dual(x0, 1)

...
"""
function xdual(x0)
    # código
    
    return Dual(x0,1)
    
end


+(a::Dual, b::Dual) = Dual(a.fun+b.fun, a.der+b.der)
+(a::Dual, b::Real) = Dual(a.fun+b,a.der)
+(a::Real, b::Dual) = Dual(a+b.fun,b.der)

-(a::Dual, b::Dual) = Dual(a.fun-b.fun, a.der-b.der)
-(a::Dual, b::Real) = Dual(a.fun-b,a.der)
-(a::Real, b::Dual) = Dual(a-b.fun,-b.der)

*(a::Dual, b::Dual) = Dual(a.fun*b.fun, a.fun*b.der+b.fun*a.der)
*(a::Dual, b::Real) = Dual(b*a.fun,b*a.der)
*(a::Real, b::Dual) = Dual(a*b.fun,a*b.der)

/(a::Dual, b::Dual) = Dual(a.fun/b.fun, a.der-(a.fun/b.fun)*b.der/b.fun)
/(a::Dual, b::Real) = Dual( a.fun/b, a.der/b)

^(a::Dual, b::Int64) = Dual(a.fun^b,b*a.fun^(b-1)*a.der)

==(a::Dual, b::Dual) = a.fun==b.fun && a.der==b.der
==(a::Dual, b::Real) = a==b.fun && 0.0==b.der
==(a::Real, b::Dual) = a.fun==b && a.der==0.0

end

In [ ]:
include("AutomDiff.jl")
using Base.Test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos

a = Dual(1, 2.0)
constante=Dual(4)
constante2=xdual(9)
constante3=xdual(8)

@testset "pruebas de operaciones para dual y xdual" begin
    @test a.fun==1.0
    @test a.der==2.0
    @test constante.fun == 4 
    @test constante.der == 0
    @test constante2.fun == 9
    @test constante2.der == 1
    @test constante3.fun == 8
    @test constante3.der == 1
    @test Dual(4,1)+Dual(9,2)==Dual(13,3)
    @test Dual(4,1)+Dual(9)==Dual(13,1)    
    @test Dual(4)+Dual(9,2)==Dual(13,2)    
    @test Dual(4,1)-Dual(9,2)==Dual(-5,-1)    
    @test Dual(4,1)-Dual(9)==Dual(-5,1) 
    @test Dual(4)-Dual(9,2)==Dual(-5,-2)  
    @test Dual(4,1)*Dual(9,2)==Dual(36,17)
    @test Dual(4,1)*Dual(9)==Dual(36,9)
    @test Dual(4)*Dual(9,2)==Dual(36,8)
    @test Dual(4,2)/Dual(2,2)==Dual(2,0)
    @test Dual(4,2)/Dual(2)==Dual(2,2)
    @test Dual(2)/Dual(2,2)==Dual(1,-1)
    @test Dual(4,2)^2==Dual(16,16)
    @test Dual(4)==Dual(4,0)
    @test Dual(4.0)==Dual(4.0,0.0)
    @test Dual(4,0)==Dual(4)
    @test Dual(5)==Dual(5,0)
    @test Dual(8,2)==Dual(8,2)
    @test Dual(2,2)-4==Dual(-2,2)
    @test Dual(2,2)+4==Dual(6,2)
    @test 4-Dual(1,1)==Dual(3,-1)
    @test 4+Dual(2,2)==Dual(6,2)
    @test Dual(2,2)*4==Dual(8,8)
    @test 4*Dual(2,2)==Dual(8,8)
    @test Dual(2,2)/4==Dual(0.5,0.5)
    @test Dual(2,1)-2==Dual(0,1)
    @test Dual(2,1)/2==Dual(1.0,0.5)
    @test xdual(2) == Dual(2,1)
    @test xdual(2.0) == Dual(2,1)
    @test f(xdual(1))==Dual(1,9)
end